TASA DE DEMANDA POTENCIAL DE CUIDADOS POR POBLACIÓN TOTAL (NINOS, NIÑAS Y ADOLESCENTES, DISCAPACITADOS Y ADULTOS MAYORES)

In [ ]:
##LIBRERIAS
librerias <- c("tidyverse",
               "zip",
               "readr",
               "srvyr")

instalar <- rownames(installed.packages())
instalar_librerias <- setdiff(librerias, instalar)

if (length(instalar_librerias) > 0) {
  install.packages(instalar_librerias)
}

lapply(librerias, library, character.only = TRUE)

print("Librerias cargadas")

In [2]:
#tiempo de espera
options(timeout = 300)  
#Descarga de archivos
url<-"https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_poblacion_csv.zip"
##Creación de directorio temporal
td<- tempdir()
# Descarga del archivo temporal
tf = tempfile(tmpdir=td, fileext=".zip")
download.file(url, tf)
unzip(tf, files="poblacion.csv", exdir=td, 
      overwrite=TRUE)
fpath=file.path(td,"poblacion.csv")
unlink(td)

In [3]:
pob <- read_csv(fpath, show_col_types = FALSE) %>%
  mutate(
    cve_ent = substr(folioviv, 1, 2),  
    pob_demandante_cuidados = case_when(
      edad <= 19 ~ "niños_niñas_adoles",
      edad >= 60 ~ "adult_may",
      disc_camin == 1 | disc_ver == 1 | disc_brazo == 1 | disc_apren == 1 |
      disc_oir == 1 | disc_vest == 1 | disc_habla == 1 | disc_acti == 1 ~ "discap",
      TRUE ~ "otros")) %>%
    filter(cve_ent == '09') %>%
    select(folioviv, foliohog, numren, upm, factor, est_dis, cve_ent, edad, disc_camin, disc_ver, disc_brazo, disc_apren, disc_oir, disc_vest, disc_habla, disc_acti, pob_demandante_cuidados)%>%
    mutate(pob_demandante_cuidados = case_when(pob_demandante_cuidados %in% c("niños_niñas_adoles", "adult_may", "discap") ~ 1, 
        TRUE ~ 0)) 
    

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


In [4]:
## Diseño muestral
pob_dm <- pob %>%
  as_survey_design(
    ids = upm,
    strata = est_dis,
    weights = factor,
    nest = TRUE)

In [5]:
tasa_demanda_cuidados <- pob_dm %>%
    group_by(pob_demandante_cuidados) %>%
    summarise(tasa_demanda = survey_total(vartype="cv"))|>
  filter(pob_demandante_cuidados==1)|>
  mutate(tasa_demanda = tasa_demanda / sum(pob$factor) * 100000) %>%
    ungroup() %>%
    select(pob_demandante_cuidados, tasa_demanda)

In [6]:
print(paste("Tasa de demanda potencial de cuidados: ", tasa_demanda_cuidados))

[1] "Tasa de demanda potencial de cuidados:  1"               
[2] "Tasa de demanda potencial de cuidados:  42491.9537485106"
